# Initial EDA on Raw UNGA dataset

Data set is taken from here: https://dataverse.harvard.edu/dataset.xhtml?persistentId=hdl:1902.1/12379

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.cluster import KMeans, k_means
from sklearn.metrics import silhouette_score
from sklearn.datasets.samples_generator import make_blobs
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
from sklearn.preprocessing import scale
%matplotlib inline
!pip install HDBSCAN
import hdbscan
from sklearn import metrics
from sklearn.cluster import DBSCAN

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
df_votes = pd.read_csv('~/Desktop/dataverse_files/UNVotes1-72.tab',sep='\t')

In [3]:
df_votes.member = df_votes.member.fillna(value=0)

In [4]:
df_votes_C = df_votes.drop(['Country', 'Countryname'], axis=1)

In [5]:
df_votes_1960 = df_votes_C[df_votes.year == 1985]

In [6]:
df_votes_1960_members = df_votes_1960[df_votes_1960.member == 1]

In [7]:
df_votes_1960_binary = df_votes_1960[(df_votes_1960.vote <= 1) | (df_votes_1960.vote == 3)]

In [8]:
df_votes_1960_binary.vote = df_votes_1960_binary.vote.replace(to_replace=3, value=0, inplace=False, limit=None, regex=False, method='pad', axis=None)

/Users/jakerattner/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [10]:
df_1960_features = df_votes_1960_binary.drop([ 'session', 'year'], axis=1)

In [11]:
df_1960_features['v_res'] = df_1960_features['rcid']*df_1960_features['vote']

In [12]:
df_1960_dummies = pd.concat([df_1960_features,pd.get_dummies(df_1960_features.v_res, prefix='res_')], axis = 1)

In [13]:
df_t = df_1960_dummies.groupby('ccode').sum()

In [14]:
df_t = df_t.drop(['v_res', 'res__0.0', 'vote','member','rcid'], axis=1)

In [15]:
features = list(df_t)

# DB Scan

In [16]:
dbscn = DBSCAN(eps = 3.2, min_samples = 15)

In [17]:
dbscn.fit(df_t[features])

DBSCAN(algorithm='auto', eps=3.2, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=15, n_jobs=1, p=None)

In [18]:
dbscn.labels_;

In [19]:
labels = dbscn.labels_

In [20]:
dbscn.core_sample_indices_ #core samples is an observation that has at least min_sample observations within its eps

array([ 71,  72,  74,  81,  92, 111, 113, 137])

In [21]:
core_samples = np.zeros_like(labels, dtype = bool)

In [22]:
core_samples;

In [23]:
core_samples[dbscn.core_sample_indices_] = True
# print(core_samples)

In [24]:
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

In [25]:
n_clusters_

2

In [26]:
print('Estimated number of clusters: %d' % n_clusters_)
# print('Homogeneity: %0.3f' % metrics.homogeneity_score(y, labels))
# print('Completeness: %0.3f' % metrics.completeness_score(y, labels))
print('Silhouette Coefficient: %0.3f' % metrics.silhouette_score(df_t[features], labels))

Estimated number of clusters: 2
Silhouette Coefficient: -0.043


# HDBSCAN

In [27]:
hdbscn = hdbscan.HDBSCAN(min_cluster_size = 10,
               gen_min_span_tree = True)

In [28]:
hdbscn.fit(df_t[features])

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
    approx_min_span_tree=True, cluster_selection_method='eom',
    core_dist_n_jobs=4, gen_min_span_tree=True, leaf_size=40,
    match_reference_implementation=False, memory=Memory(cachedir=None),
    metric='euclidean', min_cluster_size=10, min_samples=None, p=None,
    prediction_data=False)

In [29]:
hdbscn_labels = hdbscn.labels_

In [30]:
hdbscn_labels 

array([ 0,  0,  1,  1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1, -1,  1, -1,
        1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,
        1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  1,  1,  0,  1,
       -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  0,  0,  0,  0,  1,  1,  1,
        1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,
        1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,
        1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  0,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  0,  1,
        0,  1, -1,  1,  1])

In [31]:
hdbscn_n_clusters_ = len(set(hdbscn_labels)) - (1 if -1 in hdbscn_labels else 0)

In [32]:
print('Estimated number of clusters: %d' % hdbscn_n_clusters_)
# print('Homogeneity: %0.6f' % metrics.homogeneity_score(y, hdbscn_labels))
# print('Completeness: %0.6f' % metrics.completeness_score(y, hdbscn_labels))
print('Silhouette Coefficient: %0.6f' % metrics.silhouette_score(df_t[features], hdbscn_labels))

Estimated number of clusters: 2
Silhouette Coefficient: 0.363391


I can assume from all the tests I've run that this is the best mode: HDBSCAN.  This is because it both auto selects number of clusters and it performs much better than DBScan

# Kmeans

In [ ]:
# Run k-means against our features with three clusters.
k = 2
model = KMeans(n_clusters=k, max_iter=1)
model.fit(df_t[features].values)

# Check our results.
# Assign clusters back to our DataFrame.
df_t['cluster'] = model.labels_

# Get our centroids.
centroids    =  model.cluster_centers_
cc           =  pd.DataFrame(centroids)

# Set up some colors for plotting (for later).
colors = {'D':'red', 'E':'blue', 'F':'green', 'H': 'yellow'}
base_colors  =  ['r', 'g', 'b', 'y']
colors       =  [base_colors[centroid] for centroid in model.labels_]

# Update x and y of our final clusters to plot later.
fig, ax      =  plt.subplots(figsize=(7,6))

# Plot the scatter of our points with calculated centroids.
ax.scatter(df_t[features[0]], df_t[features[1]], c=colors)
ax.scatter(cc[0], cc[1], c=base_colors, s=100) # cc.index

# And our score is:
print ("Silhouette Score: ", silhouette_score(df_t[features], df_t['cluster'], metric='euclidean'))

In [ ]:
df_votes.head()

In [ ]:
df_t.reset_index(inplace=True)
df_t.head()

In [ ]:
df_votes[['ccode','Countryname']].head()

In [ ]:
df_name_index = pd.DataFrame(df_votes.groupby(['ccode', 'Countryname'])['Countryname'].count())

In [ ]:
df_name_index['count'] = df_name_index.Countryname
df_name_index = df_name_index.drop([ 'Countryname'], axis=1)
df_name_index.head()

In [ ]:
df_name_index.reset_index(inplace=True)
#df_name_index.tail(100)

In [ ]:
#df_t.join(df_name_index, how='left', on='ccode')
df_votes_1960 = pd.merge(df_t, df_name_index, on='ccode')
df_votes_1960.head()

In [ ]:
df_votes.to_csv('df_votes.csv', index=False)

In [ ]:
df_t.to_csv('df_t.csv', index=False)

In [ ]:
#let's see which countres are in the two clusters

In [ ]:
df_votes_1960[df_votes_1960.cluster == 1]['Countryname'] 

In [ ]:
 df_votes_1960.to_csv('df_votes_1960.csv', index=False)

# PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
df_PCA_1960 = df_votes_1960.drop(['count', 'Countryname'], axis=1)

In [ ]:
df_PCA_1960.head(2)

In [ ]:
#df_votes_1960.set_index('ccode')
df_PCA_1960.head(2)

In [ ]:
X = df_PCA_1960

pca = PCA()
pca = pca.fit(X)
X

In [ ]:
Z = pca.transform(X)
features_pca = ['PC'+str(i+1) for i in range(pca.n_components_)]
Z = pd.DataFrame(Z, columns=features_pca)
Z.head()

In [ ]:
print(pca.explained_variance_)

In [ ]:
abs((pca.components_))[0]

In [ ]:

print(abs((pca.components_))[0], '\n')
#print('-------------------------------------\n')



In [ ]:
X.reset_index(inplace=True)
X.head()

In [ ]:
df_PCA_1960 = df_PCA_1960.drop(['res__627.0', 'res__628.0', 'res__629.0', 'res__630.0', 'res__631.0', 
                            'res__632.0', 'res__633.0', 'res__634.0', 'res__635.0', 'res__636.0', 
                            'res__637.0', 'res__638.0', 'res__639.0', 'res__640.0', 'res__641.0', 
                            'res__642.0', 'res__643.0', 'res__644.0', 'res__645.0', 'res__646.0', 
                            'res__647.0', 'res__648.0', 'res__649.0', 'res__650.0', 'res__651.0', 
                            'res__652.0', 'res__653.0', 'res__654.0', 'res__655.0', 'res__656.0', 
                            'res__657.0', 'res__658.0', 'res__659.0', 'res__660.0', 'res__661.0', 
                            'res__662.0', 'res__663.0', 'res__664.0', 'res__665.0', 'res__666.0', 
                            'res__667.0', 'res__668.0', 'res__669.0', 'res__670.0', 'res__671.0', 
                            'res__672.0', 'res__673.0', 'res__674.0', 'res__675.0', 'res__676.0', 
                            'res__677.0', 'res__678.0', 'res__9040.0', 'res__9041.0'], axis=1)


In [ ]:
#pd.concat([X,Z])
X = X.join(Z, lsuffix='_caller', rsuffix='_other')
X

In [ ]:
df_PCA_1960 = pd.merge(X, df_PCA_1960, on='ccode')

In [ ]:
df_PCA_1960 = df_PCA_1960.drop(['res__627.0', 'res__628.0', 'res__629.0', 'res__630.0', 'res__631.0', 
                            'res__632.0', 'res__633.0', 'res__634.0', 'res__635.0', 'res__636.0', 
                            'res__637.0', 'res__638.0', 'res__639.0', 'res__640.0', 'res__641.0', 
                            'res__642.0', 'res__643.0', 'res__644.0', 'res__645.0', 'res__646.0', 
                            'res__647.0', 'res__648.0', 'res__649.0', 'res__650.0', 'res__651.0', 
                            'res__652.0', 'res__653.0', 'res__654.0', 'res__655.0', 'res__656.0', 
                            'res__657.0', 'res__658.0', 'res__659.0', 'res__660.0', 'res__661.0', 
                            'res__662.0', 'res__663.0', 'res__664.0', 'res__665.0', 'res__666.0', 
                            'res__667.0', 'res__668.0', 'res__669.0', 'res__670.0', 'res__671.0', 
                            'res__672.0', 'res__673.0', 'res__674.0', 'res__675.0', 'res__676.0', 
                            'res__677.0', 'res__678.0', 'res__9040.0', 'res__9041.0'], axis=1)


In [ ]:
df_PCA_1960 = df_PCA_1960.drop(['index_x','PC3', 'PC4', 'PC5',
       'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14',
       'PC15', 'PC16', 'PC17', 'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23',
       'PC24', 'PC25', 'PC26', 'PC27', 'PC28', 'PC29', 'PC30', 'PC31', 'PC32',
       'PC33', 'PC34', 'PC35', 'PC36', 'PC37', 'PC38', 'PC39', 'PC40', 'PC41',
       'PC42', 'PC43', 'PC44', 'PC45', 'PC46', 'PC47', 'PC48', 'PC49', 'PC50',
       'PC51', 'PC52', 'PC53', 'PC54', 'PC55', 'PC56', 'index_y', 'cluster_y'], axis=1)

In [ ]:
df_PCA_1960.head(2)

In [ ]:
fig = plt.figure(figsize=(9,7))

ax = fig.gca()
ax.scatter(Z.iloc[:,0], Z.iloc[:,1],
                   c=df_PCA_1960.cluster_x)

ax.set_xlabel('principal component 1', fontsize=16)
ax.set_ylabel('principal component 2', fontsize=16)


for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(12) 
    
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(12) 
    
ax.set_title('PC1 vs PC2\n', fontsize=20)

plt.show()

In [ ]:
df_PCA_1960.head(2)

# Junk and other imports

In [ ]:
# users[users.age == 20]
#
# Select one column from the filtered results.
# users[users.age < 20].occupation
# users[users.age < 20].occupation.value_counts()
# users[(users.age < 20) & (users.gender=='M')]
# users[users.occupation.isin(['doctor', 'lawyer'])]

In [ ]:
df_points = pd.read_csv('~/Desktop/dataverse_files/IdealpointsPublished.tab',sep='\t')
# df points is Ideal point data. Unit of analysis is the country-session 
# (which corresponds closely but not perfectly, to country-year). This 
# contains country-specific ideal point estimates. 

In [ ]:
df_points.columns

In [ ]:
df_dyadic = pd.read_csv('~/Desktop/dataverse_files/Dyadicdata.tab',sep='\t')

In [ ]:
df_dyadic.columns
# Dyadid affinity and ideal point data. Unit of analysis is the dyad-session (year). 
# This file contains distances between country ideal points as well as Affinity scores 
# (although we recommend against using these). 

In [ ]:
df_descriptions = pd.read_csv('~/documents/UN-Capstone/Dscription.csv')

In [ ]:
df_descriptions.tail(30)

In [ ]:
df_descriptions.head()

In [ ]:
df_UNAid = pd.read_csv('~/documents/UN-Capstone/net_aid.csv')

In [ ]:
df_UNAid